In [ ]:
from tkinter import ttk, filedialog
import tkinter as tk
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

class StartPage(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent, bg='#2C3E50')
        label = ttk.Label(self, text="Welcome to Transaction Insight Analytics!", font=('Century Gothic Bold', 36), foreground='#ECF0F1', background='#2C3E50')
        label.pack(pady=10, padx=10)

        csv_label = ttk.Label(self, text="Note: Only CSV files are supported.", style='FWBB2.TLabel')
        csv_label.pack(pady=10)

        load_dataset_button = ttk.Button(self, text="Load Dataset (CSV)", style='FB.TButton', command=lambda: controller.load_dataset())
        load_dataset_button.pack(pady=10)

        controller.loaded_dataset_label = ttk.Label(self, text="", style='FWBB2.TLabel')
        controller.loaded_dataset_label.pack(pady=10)

        algorithm_label = ttk.Label(self, text="Select Algorithm:", style='FWBB2.TLabel')
        algorithm_label.pack(pady=10)

        algorithms = ["Logistic Regression", "Decision Tree", "Random Forest"]
        for i, algorithm in enumerate(algorithms):
            algorithm_button = ttk.Button(self, text=algorithm, style='FB.TButton', command=lambda algo=algorithm: controller.submit_algorithm(algo))
            algorithm_button.pack(pady=10)

class LogisticRegressionAlgorithmPage(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller

        label = ttk.Label(self, text="Logistic Regression Algorithm", font=('Century Gothic Bold', 16))
        label.pack(pady=10, padx=10)

        roc_curve_button = ttk.Button(self, text="ROC Curve", command=self.display_roc_curve, style='FB.TButton')
        roc_curve_button.pack(pady=5)

        confusion_matrix_button = ttk.Button(self, text="Confusion Matrix", command=self.display_confusion_matrix, style='FB.TButton')
        confusion_matrix_button.pack(pady=5)

        metrics_button = ttk.Button(self, text="Classification Metrics", command=self.display_classification_metrics, style='FB.TButton')
        metrics_button.pack(pady=5)

        self.fig, self.ax = plt.subplots(figsize=(8, 6))
        self.canvas = FigureCanvasTkAgg(self.fig, self)
        self.canvas.get_tk_widget().pack()

    def display_roc_curve(self):
        y_prob = self.model.predict_proba(self.X_test)[:, 1]
        fpr, tpr, thresholds = roc_curve(self.y_test, y_prob)
        roc_auc = roc_auc_score(self.y_test, y_prob)

        self.ax.clear()
        self.ax.plot(fpr, tpr, color='darkorange', lw=2, label=f'AUC = {roc_auc:.2f}')
        self.ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        self.ax.set_xlabel('False Positive Rate')
        self.ax.set_ylabel('True Positive Rate')
        self.ax.set_title('ROC Curve')
        self.ax.legend(loc="lower right")

        self.canvas.draw()

    def display_confusion_matrix(self):
        Y_predicted = self.model.predict(self.X_test)
        conf_matrix = confusion_matrix(self.y_test, Y_predicted)

        self.ax.clear()
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)
        self.ax.set_title('Confusion Matrix')
        self.ax.set_xlabel('Predicted')
        self.ax.set_ylabel('Actual')

        self.canvas.draw()

    def display_classification_metrics(self):
        y_pred = self.model.predict(self.X_test)

        class_labels = ['Existing Customer', 'Attrited Customer']
        class_report = classification_report(self.y_test, y_pred, target_names=class_labels, output_dict=True)

        precision_values = [class_report[label]['precision'] for label in class_labels]
        recall_values = [class_report[label]['recall'] for label in class_labels]
        f1_values = [class_report[label]['f1-score'] for label in class_labels]

        metrics = ['Precision', 'Recall', 'F1-score']
        values = [precision_values, recall_values, f1_values]
        self.ax.clear()

        for i, metric in enumerate(metrics):
            self.ax.bar(class_labels, values[i], label=metric)

        self.ax.set_ylim(0, 1)
        self.ax.set_title('Precision, Recall, and F1-score for each class')
        self.ax.set_ylabel('Score')
        self.ax.legend()

        self.canvas.draw()

    def load_and_train_logistic_regression(self):
        url = self.controller.loaded_dataset_path
        df = pd.read_csv(url)

        drop_columns = ["CLIENTNUM", "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1", "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2"]
        df = df.drop(drop_columns, axis=1)

        df = df.dropna()
        df = pd.get_dummies(df, columns=["Gender", "Education_Level", "Marital_Status", "Income_Category", "Card_Category"], drop_first=True)

        X = df.drop("Attrition_Flag", axis=1)
        y = df["Attrition_Flag"].map({"Existing Customer": 0, "Attrited Customer": 1})

        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        self.model = LogisticRegression()
        self.model.fit(self.X_train, self.y_train)
        
class RandomForestAlgorithmPage(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller

        label = ttk.Label(self, text="Random Forest Algorithm", font=('Century Gothic Bold', 16))
        label.pack(pady=10, padx=10)

        roc_curve_button = ttk.Button(self, text="ROC Curve", command=self.display_roc_curve, style='FB.TButton')
        roc_curve_button.pack(pady=5)

        confusion_matrix_button = ttk.Button(self, text="Confusion Matrix", command=self.display_confusion_matrix, style='FB.TButton')
        confusion_matrix_button.pack(pady=5)

        metrics_button = ttk.Button(self, text="Classification Metrics", command=self.display_classification_metrics, style='FB.TButton')
        metrics_button.pack(pady=5)

        self.fig, self.ax = plt.subplots(figsize=(8, 6))
        self.canvas = FigureCanvasTkAgg(self.fig, self)
        self.canvas.get_tk_widget().pack()

    def display_roc_curve(self):
        y_prob = self.model.predict_proba(self.X_test)[:, 1]
        fpr, tpr, thresholds = roc_curve(self.y_test, y_prob)
        roc_auc = roc_auc_score(self.y_test, y_prob)

        self.ax.clear()
        self.ax.plot(fpr, tpr, color='darkorange', lw=2, label=f'AUC = {roc_auc:.2f}')
        self.ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        self.ax.set_xlabel('False Positive Rate')
        self.ax.set_ylabel('True Positive Rate')
        self.ax.set_title('ROC Curve')
        self.ax.legend(loc="lower right")

        self.canvas.draw()

    def display_confusion_matrix(self):
        Y_predicted = self.model.predict(self.X_test)
        conf_matrix = confusion_matrix(self.y_test, Y_predicted)

        self.ax.clear()
        class_labels = sorted(self.y_test.unique())  
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)
        self.ax.set_title('Confusion Matrix')
        self.ax.set_xlabel('Predicted')
        self.ax.set_ylabel('Actual')

        self.canvas.draw()

    def display_classification_metrics(self):
        y_pred = self.model.predict(self.X_test)

        class_labels = ['Existing Customer', 'Attrited Customer']
        class_report = classification_report(self.y_test, y_pred, target_names=class_labels, output_dict=True)

        precision_values = [class_report[label]['precision'] for label in class_labels]
        recall_values = [class_report[label]['recall'] for label in class_labels]
        f1_values = [class_report[label]['f1-score'] for label in class_labels]

        metrics = ['Precision', 'Recall', 'F1-score']
        values = [precision_values, recall_values, f1_values]
        self.ax.clear()

        for i, metric in enumerate(metrics):
            self.ax.bar(class_labels, values[i], label=metric)

        self.ax.set_ylim(0, 1)
        self.ax.set_title('Precision, Recall, and F1-score for each class')
        self.ax.set_ylabel('Score')
        self.ax.legend()

        self.canvas.draw()

    def load_and_train_random_forest(self):
        url = self.controller.loaded_dataset_path
        df = pd.read_csv(url)

        drop_columns = ["CLIENTNUM", "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1", "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2"]
        df = df.drop(drop_columns, axis=1)
        df = df.dropna()
        df = pd.get_dummies(df, columns=["Gender", "Education_Level", "Marital_Status", "Income_Category", "Card_Category"], drop_first=True)

        X = df.drop("Attrition_Flag", axis=1)
        y = df["Attrition_Flag"].map({"Existing Customer": 0, "Attrited Customer": 1})

        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        self.model =  RandomForestClassifier()
        self.model.fit(self.X_train, self.y_train)

class DecisionTreeAlgorithmPage(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller

        label = ttk.Label(self, text="Decision Tree Algorithm", font=('Century Gothic Bold', 16))
        label.pack(pady=10, padx=10)

        roc_curve_button = ttk.Button(self, text="ROC Curve", command=self.display_roc_curve, style='FB.TButton')
        roc_curve_button.pack(pady=5)

        confusion_matrix_button = ttk.Button(self, text="Confusion Matrix", command=self.display_confusion_matrix, style='FB.TButton')
        confusion_matrix_button.pack(pady=5)

        metrics_button = ttk.Button(self, text="Classification Metrics", command=self.display_classification_metrics, style='FB.TButton')
        metrics_button.pack(pady=5)

        self.fig, self.ax = plt.subplots(figsize=(8, 6))
        self.canvas = FigureCanvasTkAgg(self.fig, self)
        self.canvas.get_tk_widget().pack()

    def display_roc_curve(self):
        y_prob = self.model.predict_proba(self.X_test)[:, 1]
        fpr, tpr, thresholds = roc_curve(self.y_test, y_prob)
        roc_auc = roc_auc_score(self.y_test, y_prob)

        self.ax.clear()
        self.ax.plot(fpr, tpr, color='darkorange', lw=2, label=f'AUC = {roc_auc:.2f}')
        self.ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        self.ax.set_xlabel('False Positive Rate')
        self.ax.set_ylabel('True Positive Rate')
        self.ax.set_title('ROC Curve')
        self.ax.legend(loc="lower right")

        self.canvas.draw()

    def display_confusion_matrix(self):
        Y_predicted = self.model.predict(self.X_test)
        conf_matrix = confusion_matrix(self.y_test, Y_predicted)

        self.ax.clear()
        class_labels = sorted(self.y_test.unique())  
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)
        self.ax.set_title('Confusion Matrix')
        self.ax.set_xlabel('Predicted')
        self.ax.set_ylabel('Actual')

        self.canvas.draw()

    def display_classification_metrics(self):
        y_pred = self.model.predict(self.X_test)

        class_labels = ['Existing Customer', 'Attrited Customer']
        class_report = classification_report(self.y_test, y_pred, target_names=class_labels, output_dict=True)
        precision_values = [class_report[label]['precision'] for label in class_labels]
        recall_values = [class_report[label]['recall'] for label in class_labels]
        f1_values = [class_report[label]['f1-score'] for label in class_labels]

        metrics = ['Precision', 'Recall', 'F1-score']
        values = [precision_values, recall_values, f1_values]

        self.ax.clear()

        for i, metric in enumerate(metrics):
            self.ax.bar(class_labels, values[i], label=metric)

        self.ax.set_ylim(0, 1)
        self.ax.set_title('Precision, Recall, and F1-score for each class')
        self.ax.set_ylabel('Score')
        self.ax.legend()

        self.canvas.draw()

    def load_and_train_decision_tree(self):
        # Load the dataset
        url = self.controller.loaded_dataset_path
        df = pd.read_csv(url)

        drop_columns = ["CLIENTNUM", "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1", "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2"]
        df = df.drop(drop_columns, axis=1)
        df = df.dropna()
        df = pd.get_dummies(df, columns=["Gender", "Education_Level", "Marital_Status", "Income_Category", "Card_Category"], drop_first=True)

        X = df.drop("Attrition_Flag", axis=1)
        y = df["Attrition_Flag"].map({"Existing Customer": 0, "Attrited Customer": 1})

        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        self.model =  DecisionTreeClassifier()
        self.model.fit(self.X_train, self.y_train)

class MachineLearningAlgorithm(tk.Tk):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        style = ttk.Style(self)

        style.configure('BB.TFrame', background='#3b3b3b')
        style.configure('FB.TButton', foreground='#3b3b3b', font=('Century Gothic Bold', 14))

        self.title("Transaction Insight Analytics")
        self.state('zoomed')
        container = tk.Frame(self)
        container.grid(row=0, column=0, sticky='NSEW')
        container.grid_rowconfigure(0, weight=1)
        container.grid_columnconfigure(0, weight=1)
        
        self.frames = {'StartPage': StartPage(parent=container, controller=self),
                       'LogisticRegressionAlgorithmPage': LogisticRegressionAlgorithmPage(parent=container, controller=self),
                      'RandomForestAlgorithmPage': RandomForestAlgorithmPage(parent=container, controller=self),
                      'DecisionTreeAlgorithmPage': DecisionTreeAlgorithmPage(parent=container, controller=self)}
                     
        
        for frame_name, frame in self.frames.items():
            frame.grid(row=0, column=0, sticky='NSEW')

        self.show_frame("StartPage")

        self.rowconfigure(0, weight=1)
        self.columnconfigure(0, weight=1)

        self.loaded_dataset_label = ttk.Label(self, text="", style='FWBB2.TLabel')
        self.loaded_dataset_label.grid(row=3, column=0, padx=10, pady=10)

    def show_frame(self, page_name):
        frame = self.frames[page_name]
        frame.tkraise()

    def load_dataset(self):
        file_path = filedialog.askopenfilename(title="Select Dataset", filetypes=[("CSV files", "*.csv")])
        self.loaded_dataset_label.config(text=f"Loaded Dataset: {file_path}")
        self.loaded_dataset_path = file_path

        if isinstance(self.frames["LogisticRegressionAlgorithmPage"], LogisticRegressionAlgorithmPage):
            self.frames["LogisticRegressionAlgorithmPage"].load_and_train_logistic_regression()
            
        if isinstance(self.frames["RandomForestAlgorithmPage"], RandomForestAlgorithmPage):
            self.frames["RandomForestAlgorithmPage"].load_and_train_random_forest()
            
        if isinstance(self.frames["DecisionTreeAlgorithmPage"], DecisionTreeAlgorithmPage):
            self.frames["DecisionTreeAlgorithmPage"].load_and_train_decision_tree()    

    def submit_algorithm(self, selected_algorithm):
        if selected_algorithm and hasattr(self, 'loaded_dataset_path'):
            print(f"Selected Algorithm: {selected_algorithm}")
            print(f"Loaded Dataset: {self.loaded_dataset_path}")

            print(selected_algorithm)
            if selected_algorithm == "Logistic Regression":
                self.show_frame("LogisticRegressionAlgorithmPage")
                
            elif selected_algorithm == "Random Forest":
                self.show_frame("RandomForestAlgorithmPage")
                
            elif selected_algorithm == "Decision Tree":
                self.show_frame("DecisionTreeAlgorithmPage") 
            
app = MachineLearningAlgorithm()
app.mainloop()
